In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
#ref: Solving two-stage robust optimization problems using a column-and-constraint generation method

In [3]:
#define master problem

In [4]:

# Constant creation
f = [400, 414, 326]
a = [18, 25, 20]
C = [[22, 33, 24],
      [33, 23, 30],
      [20, 25, 27], ]
D = [206 + 40, 274 + 40, 220 + 40]
dl = [206, 274, 220]
du = [40, 40, 40]
k = 0  # Iterative counting

# Create model
MP = gp.Model()  # Master-problem
SP = gp.Model()  # Sub-problem(KKT)
SPSD = gp.Model()  # Sub-problem (strong duality)
# Construction of Master-problem
# addVars
y = MP.addVars(len(f), lb=0, ub=1, obj=f, vtype=GRB.INTEGER, name='y')
z = MP.addVars(len(a), lb=0, obj=a, vtype=GRB.CONTINUOUS, name='z')
#g = MP.addVars(3, lb=0, ub=1.0, name='g')
#deterministic problem
g = MP.addVars(3, lb=0, ub=0.0, name='g')
η = MP.addVar(obj=1.0, name='η')

# addConstrs
Column1 = MP.addConstrs((z[i] <= 800 * y[i] for i in range(3)), name='fixCap')
#Column4 = MP.addConstr(gp.quicksum(z[i] for i in range(3)) >= 772, name='zCap') #772=206+274+220+40*1.8
#deterministic problem
Column4 = MP.addConstr(gp.quicksum(z[i] for i in range(3)) >= 700, name='zCap') #772=206+274+220+40*1.8
Column5 = MP.addConstr(gp.quicksum(g[i] for i in range(2)) <= 1.2, name='uncertSet1')
Column6 = MP.addConstr(gp.quicksum(g[i] for i in range(3)) <= 1.8, name='uncertSet2')

#MP.Params.timeLimit = 100.0 # 等价于 MP.setParam('timeLimit', 100)
#MP.Params.InfUnbdInfo = 1  # Determines whether simplex (and crossover) will compute additional information when a model is determined to be infeasible or unbounded
#设置是否在控制台显示优化信息：
MP.Params.LogToConsole = 0
#设置线性规划的求解方法：
#MP.params.Method = 1  # 使用对偶单纯形法
MP.params.Method = 0  # 使用原始单纯形法(迭代慢，但占内存小）
#MP.params.Method = 2  # 使用内点法（gurobi称作barrier法）
#检查约束条件，模型得不到可行解时，才能用这个函数
#使用 MP.computeIIS() 检查不可行的约束条件
#MP.computeIIS()
#MP.write("MPilp.ilp")
#使用 m.feasRelax() 通过松弛最少的不可行约束条件，得到一个可行解（模型得不到可行解时，才能用这个函数）
if MP.Status == 3: # model is infeasible 
    print("Model is infeasible")
    MP.computeIIS()
    MP.write("model.ilp")
#print(MP.getConstrs())
MP.write("MPccg.lp")  # model print and visual inspection model,can open it with Notepad++
MP.optimize()  # Solve Model
LB = MP.objval  # get optimum value of model
print('Optimal value '+str(MP.ObjVal))
print('Optimal solution', end = " ")
for i in MP.getVars():
    print('%s = %g' % (i.varName, i.x), end = " ")
print()
print('LB: '+str(LB))
        

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-27
Optimal value 13000.0
Optimal solution y[0] = 1 y[1] = 0 y[2] = 0 z[0] = 700 z[1] = 0 z[2] = 0 g[0] = 0 g[1] = 0 g[2] = 0 η = 0 
LB: 13000.0


In [5]:
#define Sub-problem

In [6]:
#from MP import *

# 添加变量
Mx = np.zeros((3, 3))
Mλ = np.zeros((3))
Mπ = np.zeros((3))
for i in range(3):
    for j in range(3):
        Mx[i][j] = min(D[j], z[i].x)
        Mλ[i] = max(C[i][0], C[i][1], C[i][2])
        Mπ[i] = max(C[0][i], C[1][i], C[2][i])
# 子问题求解kkt
x = SP.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name='x')
g = SP.addVars(3, lb=0, ub=0.0, name='g')
d = [206 + 40 * g[0], 274 + 40 * g[1], 220 + 40 * g[2]]
α = SP.addVars(3, 3, vtype=GRB.BINARY, name='alpha')  #
β = SP.addVars(3, vtype=GRB.BINARY, name='beta')
γ = SP.addVars(3, vtype=GRB.BINARY, name='gamma') 
λ = SP.addVars(3, vtype=GRB.CONTINUOUS, name='lambda') #dual variable of demand
π = SP.addVars(3, vtype=GRB.CONTINUOUS, name='pi') #demand varuavke if capacity
A = [252, 0, 520]
S1 = SP.addConstrs(((gp.quicksum(x[i, j] for j in range(3))) <= z[i].x for i in range(3)), name='zCap')
S2 = SP.addConstrs(((gp.quicksum(x[i, j] for i in range(3))) >= d[j] for j in range(3)), name='demand')
S3 = SP.addConstrs(((λ[j] - π[i]) <= C[i][j] for i in range(3) for j in range(3)), name='dualConstr')
S4 = SP.addConstrs((Mx[i][j] * α[i, j] >= x[i, j] for i in range(3) for j in range(3)), name='linearX1') #complementary slackness: product-linearization of x
S5 = SP.addConstrs(
    ((C[i][j] - λ[j] + π[i]) <= (C[i][j] + Mπ[i]) * (1 - α[i, j]) for i in range(3) for j in range(3)),
    name='linearX1')
S6 = SP.addConstrs((λ[j] <= Mλ[j] * β[j] for j in range(3)), name='linearLam1') #complementary slackness: product-linearization of lambda
S7 = SP.addConstrs(((gp.quicksum(x[i, j] for i in range(3)) - d[j]) <= 40 * (1 - β[j]) for j in range(3)),
                   name='linearLam2')
S8 = SP.addConstrs((π[i] <= Mπ[i] * γ[i] for i in range(3)), name='linearPi1') #complementary slackness: product-linearization of pi
S9 = SP.addConstrs(((z[i].x - gp.quicksum(x[i, j] for j in range(3))) <= (1 - γ[i]) * z[i].x for i in range(3)),
                   name='linearPi2')
SP.addConstr(gp.quicksum(g[i] for i in range(2)) <= 1.2, name='uncertSet1')
SP.addConstr(gp.quicksum(g[i] for i in range(3)) <= 1.8, name='uncertSet2')
obj = gp.quicksum(C[i][j] * x[i, j] for i in range(3) for j in range(3))
SP.setObjective(obj, GRB.MAXIMIZE)  # set Objective is max
SP.write("SPkkt.lp")
SP.Params.LogToConsole = 0
SP.optimize()
print('Optimal value '+str(SP.ObjVal))
print('Optimal solution', end = " ")
for i in SP.getVars():
    print('%s = %g' % (i.varName, i.x), end = " ")
print()

d = [dl[i] + du[i] * g[i].x for i in range(3)]
UB = LB - η.x + SP.objval
print('UB: '+str(UB))


Optimal value 18854.0
Optimal solution x[0,0] = 206 x[0,1] = 274 x[0,2] = 220 x[1,0] = 0 x[1,1] = 0 x[1,2] = 0 x[2,0] = 0 x[2,1] = 0 x[2,2] = 0 g[0] = 0 g[1] = 0 g[2] = 0 alpha[0,0] = 1 alpha[0,1] = 1 alpha[0,2] = 1 alpha[1,0] = 0 alpha[1,1] = 0 alpha[1,2] = 0 alpha[2,0] = 0 alpha[2,1] = 0 alpha[2,2] = 0 beta[0] = 1 beta[1] = 1 beta[2] = 1 gamma[0] = 1 gamma[1] = 1 gamma[2] = 1 lambda[0] = 22 lambda[1] = 33 lambda[2] = 24 pi[0] = 0 pi[1] = 33 pi[2] = 30 
UB: 31854.0


In [7]:
"""
learn from Paper named:
Solving two-stage robust optimization problems using a column-and-constraint generation method
Author:Su XY
Time:2019-3-1
Place of creation:iPso
Revised by:CAI Yun
Time:2022-4-30
"""
#from SPKKT import *

while UB - LB > 10e-4:
    k = k + 1  # Iterative counting
    print('k: '+str(k))
    xx = MP.addVars(3, 3, lb=0, vtype=GRB.CONTINUOUS, name='x')
    Column2 = MP.addConstrs(((gp.quicksum(xx[i, j] for j in range(3))) <= z[i] for i in range(3)), name='zCap')
    Column3 = MP.addConstrs(((gp.quicksum(xx[i, j] for i in range(3))) >= d[j] for j in range(3)), name='demand')
    Column7 = MP.addConstr(gp.quicksum(C[i][j] * xx[i, j] for i in range(3) for j in range(3)) <= η, name='constrGen')
    MP.write('MPccg_' + str(k) + '.lp')
    MP.optimize()
    LB = MP.objval
    print('Optimal value '+str(MP.ObjVal))
    print('Optimal solution', end = " ")
    for i in MP.getVars():
        print('%s = %g' % (i.varName, i.x), end = " ")
    print()
    print('LB: '+str(LB))
    SP.remove(SP.getConstrs()[0:6])
    SP.remove(SP.getConstrs()[15:23])
    SP.remove(SP.getConstrs()[36:39])
    SP.remove(SP.getConstrs()[42:45])
    S1 = SP.addConstrs(((gp.quicksum(x[i, j] for j in range(3))) <= z[i].x for i in range(3)), name='zCap')
    S2 = SP.addConstrs(((gp.quicksum(x[i, j] for i in range(3))) >= d[j] for j in range(3)), name='demand')
    S7 = SP.addConstrs(((gp.quicksum(x[i, j] for i in range(3)) - d[j]) <= 40 * (1 - β[j]) for j in range(3)),
                       name='uncertSetLinear1')
    S9 = SP.addConstrs(((z[i].x - gp.quicksum(x[i, j] for j in range(3))) <= (1 - γ[i]) * z[i].x for i in range(3)),
                       name='uncertSetLinear2')
    for i in range(3):
        for j in range(3):
            Mx[i][j] = min(D[j], z[i].x)
    S4 = SP.addConstrs((Mx[i][j] * α[i, j] >= x[i, j] for i in range(3) for j in range(3)), name='maxDemand')
    SP.write('CCGkkt_'+str(k)+'.lp')
    SP.optimize()
    print('Optimal value '+str(SP.ObjVal))
    print('Optimal solution', end = " ")
    for i in SP.getVars():
        print('%s = %g' % (i.varName, i.x), end = " ")
    print()
    UB = min(UB, LB - η.x + SP.objval)  # update UB
    print('UB: '+str(UB))



k: 1
Optimal value 30536.0
Optimal solution y[0] = 1 y[1] = -0 y[2] = 1 z[0] = 220 z[1] = 0 z[2] = 480 g[0] = 0 g[1] = 0 g[2] = 0 η = 16250 x[0,0] = 0 x[0,1] = 0 x[0,2] = 220 x[1,0] = 0 x[1,1] = 0 x[1,2] = 0 x[2,0] = 206 x[2,1] = 274 x[2,2] = 0 
LB: 30536.0
Optimal value 16250.0
Optimal solution x[0,0] = 0 x[0,1] = 0 x[0,2] = 220 x[1,0] = 0 x[1,1] = 0 x[1,2] = 0 x[2,0] = 206 x[2,1] = 274 x[2,2] = 0 g[0] = 0 g[1] = 0 g[2] = 0 alpha[0,0] = 0 alpha[0,1] = 0 alpha[0,2] = 1 alpha[1,0] = 0 alpha[1,1] = 0 alpha[1,2] = 0 alpha[2,0] = 1 alpha[2,1] = 1 alpha[2,2] = 0 beta[0] = 1 beta[1] = 1 beta[2] = 1 gamma[0] = 1 gamma[1] = 1 gamma[2] = 1 lambda[0] = 20 lambda[1] = 25 lambda[2] = 27 pi[0] = 3 pi[1] = 33 pi[2] = 0 
UB: 30536.0
